# 사람인 데이터 크롤링

## 라이브러리 선언

In [9]:
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

## 함수 선언

In [13]:
def get_source(locations: list, jobs: list) -> BeautifulSoup:
    """
    지역 번호와 직무 코드를 받아 조건에 맞는 사람인 채용공고 목록 페이지를 반환하는 함수
    locations: 지역 번호 리스트
    jobs: 직무 코드 리스트
    source: 사람인 채용공고 목록 페이지, 함수 반환값
    """

    url = 'https://www.saramin.co.kr/zf_user/jobs/list/domestic?' + \
          'loc_mcd=' + '%'.join(location) + '&' + \
          'cat_kewd=' + '%'.join(jobs)
    
    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')

    return source

In [68]:
def get_recruit_dict(source: BeautifulSoup) -> dict:
    """
    사람인 채용공고 목록 페이지에서 회사 이름과 채용공고 글 번호를 묶어서 반환하는 함수
    source: 사람인 채용공고 목록 페이지
    recruit_dict: {'회사 이름': '채용공고 글 번호'}, 반환값
    """

    recruit_section = source.find('section', {'class': 'list_recruiting'})
    company_column = recruit_section.find_all('div', {'class': 'company_nm'})

    recruit_dict = dict()

    for company_value in company_column:
        try:
            company_tag = company_value.find('a', {'class': 'str_tit'})
            company_name = company_tag.get('title').replace('(주)', '')
            recruit_number = company_tag.get('href').split('idx=')[1]

            recruit_dict[company_name] = recruit_number
        except AttributeError:
            continue
    
    return recruit_dict

In [65]:
def get_recruit_page(idx: str) -> BeautifulSoup:
    """
    사람인 채용공고 글 번호를 받아 채용공고 페이지를 반환
    source: 사람인 채용공고 목록 페이지
    recruit_dict: {'회사 이름': '채용공고 글 번호'}, 반환값
    """

    url = 'https://www.saramin.co.kr/zf_user/jobs/relay/view?' + \
           'rec_idx=' + idx
    
    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')

    return source

In [78]:
def select_company(original_dict: dict) -> tuple:
    """
    딕셔너리에서 하나의 키값을 선택해서 해당 키와 값을 반환하는 함수
    recruit_dict: {'회사 이름': '채용공고 글 번호'}
    select_page: {'인덱스': ('회사 이름', '채용공고 글 번호')}, 반환값
    """
    
    recruit_dict = original_dict.copy()
    index, selection = 1, None

    while True:
        select_page = dict()

        try:
            for i in range(10):
                select_page[index] = recruit_dict.popitem()
                index += 1
        except KeyError:
            pass

        for idx, company in select_page.items():
            print(f'{idx}. {company[0]}')
        
        if recruit_dict:
            print('다음 페이지로 이동(n)')
            selection = input('목록을 선택해주세요. ')
            if selection == 'n':
                continue
            return select_page[int(selection)]
        else:
            print('마지막 페이지 입니다.')
            selection = input('목록을 선택해주세요. ')
            return select_page[int(selection)]

In [113]:
def crawl_saramin():
    """
    사람인 크롤링 실행 함수
    현재 반환값: ('회사 이름', '채용공고 글 번호')
    """

    location_dict = {'서울': '101000', '경기': '102000'}
    job_dict = {'인공지능': '181', '데이터분석가': '2C82'}

    locations = input('지역을 입력하세요. ').split()
    jobs = input('직업을 입력하세요. ').split()

    try:
        locations = [location_dict[x] for x in locations]
        jobs = [job_dict[x] for x in jobs]

        source = get_source(locations, jobs)
    except KeyError or ConnectionError:
        raise Exception('지역이나 직업이 올바르지 않습니다.')

    recruit_dict = get_recruit_dict(source)

    try:
        selection = select_company(recruit_dict)
        print(selection)
    except KeyError or ValueError:
        raise Exception('잘못된 번호를 입력하셨습니다.')
    
    # 작성 중

## 데이터 크롤링

In [110]:
try:
    crawl_saramin()
except Exception as e:
    print(e)
    print('다시 실행시켜 주세요.')

1. 인피닉
2. 사회복지법인삼성생명공익재단
3. 서울아산병원
4. ㈜더아이엠씨
5. 주식회사 투그램시스템즈
6. 알세미
7. 베스핀글로벌
8. 뤼이드
9. 풀무원
10. 알체라
다음 페이지로 이동(n)
('인피닉', '42688576')


## 테스트 코드

In [16]:
locations = ['101000']
jobs = ['181']

source = get_source(locations, jobs)
# source

In [64]:
recruit_dict = get_recruit_dict(source)
# company_dict

In [114]:
#get_recruit_page(recruit_dict['인라이플'])